In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from transformer_lens import HookedTransformer

from unlearning.tool import get_hf_model
from unlearning.feature_activation import get_forget_retain_data, tokenize_dataset, get_feature_activation_sparsity, get_top_features
from unlearning.jump_relu import load_gemma2_2b_sae
from unlearning.intervention import scaling_intervention
from unlearning.metrics import calculate_MCQ_metrics, get_loss_added_hf, create_df_from_metrics, generate_ablate_params_list

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle

In [4]:
model = HookedTransformer.from_pretrained('google/gemma-2-2b-it')
base_model = HookedTransformer.from_pretrained('google/gemma-2-2b')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b into HookedTransformer


In [20]:
layer = 3
sae = load_gemma2_2b_sae(layer=layer)

Found SAE with l0=59 at path google/gemma-scope-2b-pt-res/layer_3/width_16k/average_l0_59/params.npz


In [21]:
from sae.activation_store import ActivationsStore

sae.cfg.dataset = "Skylion007/openwebtext"
sae.cfg.n_batches_in_store_buffer = 8

activation_store = ActivationsStore(sae.cfg, model, create_dataloader=False)

In [22]:
activation_store.iterable_dataset = iter(activation_store.dataset)            

n_batches = 100
sae_losses = torch.zeros(n_batches)
ori_losses = torch.zeros(n_batches)

for i in range(n_batches):
    tokens = activation_store.get_batch_tokenized_data()
    sae_loss, ori_loss = sae.get_test_loss(tokens, model)
    sae_losses[i] = sae_loss
    ori_losses[i] = ori_loss
    
print(sae_losses.mean(), ori_losses.mean())



tensor(4.1257) tensor(3.2397)


In [23]:
it_model_results = (sae_losses.mean(), ori_losses.mean())

In [24]:
it_model_results[0] - it_model_results[1]

tensor(0.8860)

In [25]:

activation_store.iterable_dataset = iter(activation_store.dataset)            

n_batches = 100
sae_losses = torch.zeros(n_batches)
ori_losses = torch.zeros(n_batches)

for i in range(n_batches):
    tokens = activation_store.get_batch_tokenized_data()
    sae_loss, ori_loss = sae.get_test_loss(tokens, base_model)
    sae_losses[i] = sae_loss
    ori_losses[i] = ori_loss
    
print(sae_losses.mean(), ori_losses.mean())

tensor(3.8411) tensor(2.9781)


In [26]:
sae_losses.mean() - ori_losses.mean()

tensor(0.8631)

In [27]:
tokens.shape

torch.Size([32, 128])